In [27]:
#Импорт либ
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [28]:
#Все данные со споти дева
cid = '02d33d68c8724a3fb5209f420a3e4d1a'
secret = '7e1ecf62485a4fec84700590a1453e9f'
redirect_uri='http://localhost:7777/callback'
username = '9xg1dh4sogilprh9kw0cdw3wi'

In [29]:
#Авторизация с данными в споти и необходимыми скоупами
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [30]:
# Эту функцию скопипастил
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Запихиваем это всё в датафрэйм
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

###Получение всех песен


Получение всех песен с спотифая, занимает очень много времени и я уже это сделал так что закомментил

In [31]:
# # Getting playlist IDs from each of Spotify's playlists
# playlists = sp.user_playlists('spotify')
# spotify_playlist_ids = []
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#         spotify_playlist_ids.append(playlist['uri'][-22:])
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None
# spotify_playlist_ids[:20]

In [32]:
# len(spotify_playlist_ids)

### Получение самых прослушиваемых песен с аккаунта.

In [33]:

def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        ids.append(track['id'])
    return

In [34]:

def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [35]:
#Датасет
df = pd.read_csv('data/playlist_songs.csv')
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,Leave The Door Open,Bruno Mars,2021-03-05,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,We're Good,Future Nostalgia (The Moonlight Edition),Dua Lipa,2021-02-11,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,Hold On,Hold On,Justin Bieber,2021-03-05,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,Astronaut In The Ocean,Masked Wolf,2021-01-06,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [36]:
# Убираем лишние колонки
df = df.drop(columns=['name', 'album', 'artist', 'release_date'])
df.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [37]:
# Убираем повторяющиеся
df = df.drop_duplicates(subset=['track_id'])
df['track_id'].value_counts()

4BIej0swGWja46j5B7l4s1    1
0PspNrE4MSjr1JeB4OW0jn    1
1Pioq0rEvqUxt7yuud1OKm    1
5yGAyupR8TXCUq3MPb7xiK    1
60a0Rd6pjrkxjPbaKzXjfq    1
                         ..
5BOBHIBuzvQuIYL1E1nDzl    1
0LTSNmOLBt25GMjHlxp9OR    1
28as0lgulKikhNu8TZeb66    1
0iXQ0GWaYAC4XMivDRYR5Y    1
17a3cv3ra2wKlstmzELkP6    1
Name: track_id, Length: 8883, dtype: int64

## Getting user's favorite tracks

In [38]:
# Топ 50 песен пользователя
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [39]:
# Конвертируем в формат датасета
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Taunt,Are You Alright?,0r3QJbwBuR9UL0ai5qDTw7,Lovejoy,133779,78
1,One Day,Are You Alright?,0IUprcNjE2hDtlvY8rq2W9,Lovejoy,144616,81
2,Internet Ruined Me,Internet Ruined Me,2Q1s11otlyEtj80OgsqDFz,Wilbur Soot,255600,73
3,Your New Boyfriend,Your New Boyfriend,4vLBnQtece15fFhqWxZvJP,Wilbur Soot,239667,80
4,Roadtrip,Roadtrip,4Agtk2MrapdZAVN7v6PuFO,Dream,133661,78


In [40]:

%%time
# Собираем все параметры любимых песен
fav_tracks = []
for track in df_favourite['track_id']:
    try:  
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass

Wall time: 7.75 s


In [41]:
# Создаем датасет
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0r3QJbwBuR9UL0ai5qDTw7,Taunt,Are You Alright?,Lovejoy,2021-05-08,133779,78,0.476,0.01570,0.834,0.000034,0.2630,-9.343,0.0951,186.844,4
1,0IUprcNjE2hDtlvY8rq2W9,One Day,Are You Alright?,Lovejoy,2021-05-08,144616,81,0.332,0.00696,0.737,0.000000,0.0519,-8.572,0.0349,183.333,4
2,2Q1s11otlyEtj80OgsqDFz,Internet Ruined Me,Internet Ruined Me,Wilbur Soot,2020-05-25,255600,73,0.726,0.72400,0.371,0.002990,0.1320,-14.008,0.0623,160.240,4
3,4vLBnQtece15fFhqWxZvJP,Your New Boyfriend,Your New Boyfriend,Wilbur Soot,2020-12-11,239667,80,0.546,0.38200,0.453,0.000039,0.1060,-13.040,0.1210,104.532,4
4,4Agtk2MrapdZAVN7v6PuFO,Roadtrip,Roadtrip,Dream,2021-02-05,133661,78,0.459,0.31600,0.581,0.000000,0.1240,-6.575,0.0713,176.195,4


In [42]:
# Опять сносим колонки
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0r3QJbwBuR9UL0ai5qDTw7,133779,78,0.476,0.01570,0.834,0.000034,0.2630,-9.343,0.0951,186.844,4
1,0IUprcNjE2hDtlvY8rq2W9,144616,81,0.332,0.00696,0.737,0.000000,0.0519,-8.572,0.0349,183.333,4
2,2Q1s11otlyEtj80OgsqDFz,255600,73,0.726,0.72400,0.371,0.002990,0.1320,-14.008,0.0623,160.240,4
3,4vLBnQtece15fFhqWxZvJP,239667,80,0.546,0.38200,0.453,0.000039,0.1060,-13.040,0.1210,104.532,4
4,4Agtk2MrapdZAVN7v6PuFO,133661,78,0.459,0.31600,0.581,0.000000,0.1240,-6.575,0.0713,176.195,4


In [43]:
#Убираем дупликаты
df_fav['track_id'].value_counts()

64lsIF5pw0sJY0gV5kz0RN    1
1BoIsD3EzJHnfZkedCznNr    1
0KKkJNfGyhkQ5aFogxQAPU    1
3IxMaULfjq4IT2IN6v54PB    1
5PVPkjFKLElXpshtXtXIFc    1
0r3QJbwBuR9UL0ai5qDTw7    1
5BNAzE37hmpyCx0hyrgDqJ    1
7tOcPDj3vyopZ404pY6UuP    1
0HaNrvszmWl5vIwKuADD5Z    1
0zePbRMJ9sd7wsZRlbPQua    1
0IUprcNjE2hDtlvY8rq2W9    1
6oPgMKfRAxf8sLOYnIw26v    1
2RzlTcvW7Um8GiTTNWhxcr    1
3HEvzEwP0Jg0hEszE6P8x4    1
2WYs5LxOZfEyURXu7V0O1a    1
4kdjHU5lzooUP7kkzQqX0Q    1
2rUHJ1o9qj44fR0vTugDAx    1
1Ta5qcEWkJxlGLBbZZhas1    1
4UG8Logkq92VZuoHcqk0HO    1
0QUgSlWpeW7VCkLmrc58RU    1
7JDa53N0bUwBho5Xt0encA    1
2zkSDE5xKRuNnj0h1qFmYT    1
6HTVZeVQ9J6Uiq6tHESxU9    1
7Eta1k8UzwooyJCB9stY1d    1
4FQhyNbmwSKX4S2DiDOt5k    1
11zf7m4vw9Ze7cer9Nyhk1    1
6ShHxD0hoJSXajmcpQ5Ri8    1
7gHkDNieGzYmyuC7E1Xbxw    1
475bzi2tQdQFa5rUmHfoEM    1
4pUCPPY3BImBawPkAWtNhj    1
05k3Cp3lddjoH3ZEKwn5s3    1
1TAC5SANfK7l6wGHtUQ7MX    1
1pNUmVxDiE8t6P1XxcZAv8    1
2Q1s11otlyEtj80OgsqDFz    1
0tPAksKbZTl8VLslZErQnE    1
33datjnH65fWAuMHlt9r

In [44]:
# Создаём колонку "любимые" для классификации
df_fav['favorite'] = 1
df['favorite'] = 0 

In [45]:
# Сравниваем колонки двух датасетов
df.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## Подготовка к созданию модели

In [46]:

print(df.shape)
print(df_fav.shape)

(8883, 13)
(50, 13)


In [47]:
# Объединяем датасет всех песен с любимыми
combined = pd.concat([df, df_fav])
combined.shape

(8933, 13)

In [48]:
combined.favorite.value_counts()

0    8883
1      50
Name: favorite, dtype: int64

In [49]:
# Датафрэйм с любимыми песнями
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,0r3QJbwBuR9UL0ai5qDTw7,133779,78,0.476,0.01570,0.834,0.000034,0.2630,-9.343,0.0951,186.844,4,1
1,0IUprcNjE2hDtlvY8rq2W9,144616,81,0.332,0.00696,0.737,0.000000,0.0519,-8.572,0.0349,183.333,4,1
2,2Q1s11otlyEtj80OgsqDFz,255600,73,0.726,0.72400,0.371,0.002990,0.1320,-14.008,0.0623,160.240,4,1
3,4vLBnQtece15fFhqWxZvJP,239667,80,0.546,0.38200,0.453,0.000039,0.1060,-13.040,0.1210,104.532,4,1
4,4Agtk2MrapdZAVN7v6PuFO,133661,78,0.459,0.31600,0.581,0.000000,0.1240,-6.575,0.0713,176.195,4,1


In [50]:
# Убираем любимые песни с плэйлиста
df = combined.loc[combined['favorite'] != 1]
df.shape

(8883, 13)

In [51]:
df_fav.shape

(50, 13)

In [52]:
# Сохраняем в csv для создания модели
df.to_csv('encoded_playlist_songs.csv', index=False)
df_fav.to_csv('favorite_songs.csv', index=False)